In [ ]:
import torch
from sentence_transformers import InputExample, losses, util
from sentence_transformers import CrossEncoder
import sentence_transformers.cross_encoder.evaluation
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch import optim


Load Data Folder - Where Main Data Files are

In [ ]:
data_folder = '/home/datasets/'
fine_tune_data = pd.read_csv(data_folder+ "tuneCrossEncoder.csv", encoding='utf-8')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") ## specify the GPU id's, GPU id's start from 0.

Load model

In [ ]:
model = CrossEncoder('distilbert-base-multilingual-cased', num_labels = 1)

<h3>Finetune the cross-encoder</h3>
Then save the model on folder, for reloading next time.

In [ ]:
train_examples = []

fine_tune_data = fine_tune_data.sample(frac=1, random_state=2319).reset_index(drop=True)

#Define your train examples.
for i in range(len(fine_tune_data)-1000):
    train_examples.append(InputExample(texts=[fine_tune_data.iloc[i,0], fine_tune_data.iloc[i,1]], label= int(fine_tune_data.iloc[i,2])))


evaluation_data = fine_tune_data.loc[len(fine_tune_data)-1000:len(fine_tune_data), ['Sa', 'Sb']]
labels_evaluation = fine_tune_data.iloc[len(fine_tune_data)-1000:len(fine_tune_data),2]

sentence_pares = []
for i in (range(1000)):
    sentence_pares.append([evaluation_data.iloc[i,0], evaluation_data.iloc[i,1]])


evaluator =  sentence_transformers.cross_encoder.evaluation.CEBinaryClassificationEvaluator(
                sentence_pairs = sentence_pares,
                labels = labels_evaluation.tolist(),
                name = 'output',
                write_csv = True
                )

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = 16) 

output_path = '/home/cross_encoder tuned'

model.fit(train_dataloader = train_dataloader, 
            evaluator=evaluator, 
            epochs=15, 
            activation_fct = torch.nn.Sigmoid(),
            scheduler = 'WarmupLinear', # --> default <=> learning rate is increased from 0 up to the maximal learning rate.
            warmup_steps=100, # After these many training steps, the learning rate is decreased linearly back to zero.
            optimizer_class= optim.AdamW,
            optimizer_params={'lr': 1e-5},
            weight_decay= 0.01, 
            output_path=output_path,
            save_best_model=True,
            show_progress_bar=True,
            use_amp=True, # mixed precision for faster training
            )